In [ ]:
 !pip install transformers[torch]
!pip install sentencepiece
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 4.1 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 596 kB 65.2 MB/s 
     |████████████████████████████████| 6.6 MB 56.3 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 4.3 MB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 4.3 MB/s 
     |████████████████████████████████| 1.1 MB 67.5 MB/s 
     |████████████████████████████████| 86 kB 6.7 MB/s 
     |████████████████████████████████| 140 kB 74.2 MB/s 
     |████████████████████████████████|

In [ ]:
from datasets import load_dataset,DatasetDict,Dataset
# from datasets import 
from transformers import AutoTokenizer
import numpy as np
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.utils.class_weight import compute_class_weight
import torch

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')
print('Loading a tokenizer')

Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/455k [00:00<?, ?B/s]

Loading a tokenizer


In [ ]:

webis_train = "https://ml-coding-test.s3.eu-west-1.amazonaws.com/webis_train.csv"
webis_test = "https://ml-coding-test.s3.eu-west-1.amazonaws.com/webis_test.csv"
df_train = pd.read_csv(webis_train)
df_test = pd.read_csv(webis_test)

# Filter only importante variables
df_train = df_train[['postText', 'truthClass']]
df_test = df_test[['postText', 'truthClass']]

# Remove empty rows
df_train['postText'].replace('', np.nan, inplace=True)
df_train.dropna(subset=['postText'], inplace=True)

df_test['postText'].replace('', np.nan, inplace=True)
df_test.dropna(subset=['postText'], inplace=True)


df_train['truthClass'] = pd.factorize(df_train['truthClass'])[0]
df_test['truthClass'] = pd.factorize(df_test['truthClass'])[0]

df_train, df_valid = train_test_split(df_train, test_size = 0.2, random_state = 42, 
                                  stratify = df_train['truthClass'])

webis_total = DatasetDict()

webis_total['train'] = Dataset.from_pandas(df_train)
webis_total['validation'] = Dataset.from_pandas(df_valid)
webis_total['test'] = Dataset.from_pandas(df_test)

weight = torch.FloatTensor(compute_class_weight(class_weight="balanced", classes=[0, 1], y=df_train["truthClass"])).to(torch.device('cuda'))

  
def preprocess_function_title_only_classification(examples):
    model_inputs = tokenizer(examples['postText'], padding="longest", truncation=True, max_length=25)
      
    model_inputs['labels'] = examples['truthClass']

    return model_inputs

webis_total = webis_total.map(preprocess_function_title_only_classification, batched=True)


print(webis_total["train"][0].keys())
print(webis_total["train"][0])


print("weight:", weight)
weight = torch.FloatTensor(compute_class_weight(class_weight="balanced", classes=[0, 1], y=df_train["truthClass"])).to(torch.device('cuda'))
WEIGHT = torch.cuda.FloatTensor([weight[1]/weight[0]])
print("weight for clickbait:", WEIGHT)

/usr/local/lib/python3.7/dist-packages/pandas/core/generic.py:6619: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return self._update_inplace(result)
/usr/local/lib/python3.7/dist-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return func(*args, **kwargs)
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/19 [00:00<?, ?ba/s]

dict_keys(['postText', 'truthClass', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'])
{'postText': 'Parents reject claim Aboriginal man tried to kill himself in prison', 'truthClass': 0, '__index_level_0__': 10327, 'input_ids': [101, 3008, 15454, 4366, 9757, 2158, 2699, 2000, 3102, 2370, 1999, 3827, 102, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'labels': 0}
weight: tensor([0.6597, 2.0656], device='cuda:0')
weight for clickbait: tensor([3.1312], device='cuda:0')


In [ ]:
import torch
from transformers import BertModel, BertConfig, PretrainedConfig, PreTrainedModel, AutoModel, AutoConfig
from typing import List, Optional, Tuple, Union
from transformers.modeling_outputs import TokenClassifierOutput,SequenceClassifierOutput
from torch.nn import BCEWithLogitsLoss, CrossEntropyLoss, MSELoss, BCELoss
import torch.nn as nn


class ClickbaitConfig(PretrainedConfig):
    def __init__(
        self,
        model_type: str = "bert",
        pretrained_model: str = "bert-base-uncased",
        num_labels: int = 2,
        dropout: float = 0.1,
        inner_dim1: int = 256,
        inner_dim2: int = 32, 
        max_length: int = 512,
        load_pretrained: bool = True,
        freeze_bert: bool = True,
        **kwargs
    ):
        super(ClickbaitConfig, self).__init__(num_labels=num_labels, **kwargs)
        self.model_type = model_type
        self.pretrained_model = pretrained_model
        self.dropout = dropout
        self.inner_dim1 = inner_dim1
        self.inner_dim2 = inner_dim2
        self.max_length = max_length
        self.load_pretrained = load_pretrained
        self.freeze_bert = freeze_bert


In [ ]:
class BertClickbaitClassifier(PreTrainedModel):
    """
      Taken and extended from BertforSequenceClassification : https://github.com/huggingface/transformers/blob/v4.19.2/src/transformers/models/bert/modeling_bert.py#L1508
    """
    config_class = ClickbaitConfig
    def __init__(self, config: ClickbaitConfig):
        super(BertClickbaitClassifier, self).__init__(config)
        self.num_labels = config.num_labels
        self.config = config
        # self.bert_config = BertConfig.from_pretrained(config.pretrained_model)
        self.bert_config = AutoConfig.from_pretrained(config.pretrained_model)

        # self.bert = BertModel(self.bert_config)
        self.bert = AutoModel.from_pretrained(config.pretrained_model, config=self.bert_config)
        if config.load_pretrained:
            print("Load pretrained weights from {}".format(config.pretrained_model))
            self.bert = self.bert.from_pretrained(config.pretrained_model)
        if config.freeze_bert:
            print("Freeze weights in the BERT model. Just the classifier will be trained")
            for param in self.bert.parameters():
                param.requires_grad = False

        self.linear_1 = nn.Linear(self.bert.config.hidden_size, config.inner_dim1)
        self.dropout_1 = nn.Dropout(config.dropout) 
        self.relu_1 = nn.ReLU()
        self.dropout_2 = nn.Dropout(config.dropout)
        self.linear_2 = nn.Linear(config.inner_dim1, config.inner_dim2)
        self.relu_2 = nn.ReLU()
        self.dropout_3 = nn.Dropout(config.dropout)
        self.classifier = nn.Linear(config.inner_dim2, config.num_labels)
        self.sigmoid = nn.Sigmoid()


    def forward(
        self,
        input_ids: Optional[torch.Tensor] = None,
        attention_mask: Optional[torch.Tensor] = None,
        token_type_ids: Optional[torch.Tensor] = None,
        position_ids: Optional[torch.Tensor] = None,
        head_mask: Optional[torch.Tensor] = None,
        inputs_embeds: Optional[torch.Tensor] = None,
        labels: Optional[torch.Tensor] = None,
        output_attentions: Optional[bool] = None,
        output_hidden_states: Optional[bool] = None,
        return_dict: Optional[bool] = None,
    ) -> Union[Tuple[torch.Tensor], SequenceClassifierOutput]:
        r"""
        labels (`torch.LongTensor` of shape `(batch_size,)`, *optional*):
            Labels for computing the sequence classification/regression loss. Indices should be in `[0, ...,
            config.num_labels - 1]`. If `config.num_labels == 1` a regression loss is computed (Mean-Square loss), If
            `config.num_labels > 1` a classification loss is computed (Cross-Entropy).
        """
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict

        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids,
            position_ids=position_ids,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
            return_dict=return_dict,
        )

        output = outputs[0][:,0,:]

        x = self.dropout_1(output)
        x = self.linear_1(x)
        x = self.relu_1(x)
        x = self.dropout_2(x)
        x = self.linear_2(x)
        x = self.relu_2(x)
        x = self.dropout_3(x)

        logits = self.classifier(x)
        logits = self.sigmoid(logits)

        loss = None
        if labels is not None:
            loss_fct = BCELoss(weight=WEIGHT)
            labels = 1.0*labels
            loss = loss_fct(logits.view(-1), labels.view(-1))
        if not return_dict:
            output = (logits,) + outputs[2:]
            return ((loss,) + output) if loss is not None else output

        return SequenceClassifierOutput(
            loss=loss,
            logits=logits
        )

In [ ]:
import torch
from transformers import BertConfig

# config = ClickbaitConfig(
#     model_type="bert",
#     pretrained_model="bert-base-uncased",
#     num_labels=1,
#     dropout=0.2,
#     inner_dim1=256,
#     inner_dim2=32, 
#     max_length=25,
#     load_pretrained=True,
#     freeze_bert=False,
# )

roberta_config = ClickbaitConfig(
    model_type="bert",
    pretrained_model="roberta-base",
    num_labels=1,
    dropout=0.2,
    inner_dim1=256,
    inner_dim2=32, 
    max_length=25,
    load_pretrained=True,
    freeze_bert=False,
)

In [ ]:
# model = BertClickbaitClassifier(config=config)
model = BertClickbaitClassifier(config=roberta_config).to('cuda')

loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "_name_or_path": "roberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/roberta-base/r

Load pretrained weights from roberta-base


loading configuration file https://huggingface.co/roberta-base/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/733bade19e5f0ce98e6531021dd5180994bb2f7b8bd7e80c7968805834ba351e.35205c6cfc956461d8515139f0f8dd5d207a2f336c0c3a83b4bc8dca3518e37b
Model config RobertaConfig {
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://huggingface.co/roberta-base/resolve/main/pytorch_model.bin from 

In [ ]:
import numpy as np
from datasets import load_metric
from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, precision_score, recall_score

def compute_metrics(eval_pred):
    preds = eval_pred.predictions[0] if isinstance(eval_pred.predictions, tuple) else eval_pred.predictions
    labels = eval_pred.label_ids
    predictions = (preds > 0.5).astype(int)
    results = {
        'f1' : f1_score(labels, predictions),
        'accuracy' : accuracy_score(labels, predictions), 
        'precision' : precision_score(labels, predictions), 
        'recall' : recall_score(labels, predictions),
        'balanced_accuracy' : balanced_accuracy_score(labels, predictions), 
    }
    return results
    

In [ ]:
from transformers import Trainer
from transformers import TrainingArguments
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers import AdamW,get_scheduler
import torch
from datasets import load_metric

output_path = "drive/MyDrive/nlp_lss_data/"

debug = False

training_args = TrainingArguments(
    # output_dir=output_path+'clickbait_classification_maxlen25',           # output directory
    output_dir=output_path+'roberta_clickbait_classification_maxlen25',           # output directory
    evaluation_strategy ="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=1,
    warmup_steps=100,
    num_train_epochs=6,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    report_to="none"
)
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=webis_total["train"].shuffle(seed=42).select(range(50)) if debug else webis_total["train"].shuffle(seed=42),  
    eval_dataset=webis_total["validation"].shuffle(seed=42).select(range(10)) if debug else webis_total["validation"].shuffle(seed=42),
    compute_metrics=compute_metrics,
    tokenizer=tokenizer
)

PyTorch: setting up devices


In [ ]:
import os
os.environ["WANDB_DISABLED"] = "true"
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertClickbaitClassifier.forward` and have been ignored: truthClass, __index_level_0__, postText. If truthClass, __index_level_0__, postText are not expected by `BertClickbaitClassifier.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 15587
  Num Epochs = 6
  Instantaneous batch size per device = 64
  Total train batch size (w. parallel, distributed & accumulation) = 64
  Gradient Accumulation steps = 1
  Total optimization steps = 1464


Epoch,Training Loss,Validation Loss,F1,Accuracy,Precision,Recall,Balanced Accuracy
1,No log,1.688085,0.000000,0.758019,0.000000,0.000000,0.500000
2,No log,1.496791,0.535053,0.787272,0.567857,0.505832,0.691474
3,1.655600,1.531505,0.052367,0.758532,0.520000,0.027572,0.509724
4,1.655600,1.399927,0.568828,0.805491,0.613497,0.530223,0.711794
5,1.491500,1.378389,0.585740,0.819605,0.659151,0.527041,0.720020
6,1.491500,1.367291,0.576478,0.821658,0.677650,0.501591,0.712711


The following columns in the evaluation set don't have a corresponding argument in `BertClickbaitClassifier.forward` and have been ignored: truthClass, __index_level_0__, postText. If truthClass, __index_level_0__, postText are not expected by `BertClickbaitClassifier.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3897
  Batch size = 1
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to drive/MyDrive/nlp_lss_data/roberta_clickbait_classification_maxlen25/checkpoint-244
Configuration saved in drive/MyDrive/nlp_lss_data/roberta_clickbait_classification_maxlen25/checkpoint-244/config.json
Model weights saved in drive/MyDrive/nlp_lss_data/roberta_clickbait_classification_maxlen25/ch

TrainOutput(global_step=1464, training_loss=1.4883061393362578, metrics={'train_runtime': 523.0026, 'train_samples_per_second': 178.817, 'train_steps_per_second': 2.799, 'total_flos': 1204353585477900.0, 'train_loss': 1.4883061393362578, 'epoch': 6.0})

# Validation

In [ ]:
trainer.evaluate()

The following columns in the evaluation set don't have a corresponding argument in `BertClickbaitClassifier.forward` and have been ignored: truthClass, __index_level_0__, postText. If truthClass, __index_level_0__, postText are not expected by `BertClickbaitClassifier.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 3897
  Batch size = 1


{'epoch': 6.0,
 'eval_accuracy': 0.8527072106748781,
 'eval_balanced_accuracy': 0.7638737416634418,
 'eval_f1': 0.6603550295857988,
 'eval_loss': 1.0726171731948853,
 'eval_precision': 0.7469879518072289,
 'eval_recall': 0.591728525980912,
 'eval_runtime': 39.8656,
 'eval_samples_per_second': 97.753,
 'eval_steps_per_second': 97.753}

# Test

In [ ]:
y_test = trainer.predict(webis_total["test"])
y_pred = (y_test.predictions > 0.5).astype(int).ravel()

from sklearn.metrics import classification_report
print(classification_report(y_test.label_ids, y_pred, target_names = ["No-clickbait", "Clickbait"]))


The following columns in the test set don't have a corresponding argument in `BertClickbaitClassifier.forward` and have been ignored: truthClass, __index_level_0__, postText. If truthClass, __index_level_0__, postText are not expected by `BertClickbaitClassifier.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 18913
  Batch size = 1


              precision    recall  f1-score   support

No-clickbait       0.89      0.94      0.91     14455
   Clickbait       0.75      0.63      0.69      4458

    accuracy                           0.86     18913
   macro avg       0.82      0.78      0.80     18913
weighted avg       0.86      0.86      0.86     18913



In [ ]:
dataset_base = "drive/MyDrive/nlp_lss_data/"
# dataset_base = "/toshiba/nlp_lss_data/"

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers[torch]
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 346 kB 5.0 MB/s 
     |████████████████████████████████| 212 kB 16.5 MB/s 
     |████████████████████████████████| 1.1 MB 68.8 MB/s 
     |████████████████████████████████| 140 kB 53.7 MB/s 
     |████████████████████████████████| 86 kB 6.4 MB/s 
     |████████████████████████████████| 127 kB 59.8 MB/s 
     |████████████████████████████████| 94 kB 3.8 MB/s 
     |████████████████████████████████| 144 kB 72.5 MB/s 
     |████████████████████████████████| 271 kB 60.3 MB/s 
     |████████████████████████████████| 112 kB 76.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: dill
    Found existing installation: dill 0

In [ ]:
from transformers import AutoModel, AutoTokenizer

classifier_model_path = "drive/MyDrive/nlp_lss_data/clickbait_classification_maxlen25/checkpoint-1464"

#Loading classifier
classifier_tokenizer = AutoTokenizer.from_pretrained(classifier_model_path, load_best_model_at_end=True)
classifier_model = BertClickbaitClassifier.from_pretrained(classifier_model_path).to('cpu')


You passed along `num_labels=2` with an incompatible id to label map: {'0': 'LABEL_0'}. The number of labels wil be overwritten to 1.


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/420M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Load pretrained weights from bert-base-uncased


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
classifier_model = classifier_model.to('cuda')

In [ ]:
!wget https://zenodo.org/record/5530410/files/clickbait17-test-170720.zip?download=1

--2022-06-06 16:37:24--  https://zenodo.org/record/5530410/files/clickbait17-test-170720.zip?download=1
Resolving zenodo.org (zenodo.org)... 137.138.76.77
Connecting to zenodo.org (zenodo.org)|137.138.76.77|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 934724312 (891M) [application/octet-stream]
Saving to: ‘clickbait17-test-170720.zip?download=1’

clickbait17-test-17 100%[===================>] 891.42M  22.6MB/s    in 37s     

2022-06-06 16:38:02 (24.4 MB/s) - ‘clickbait17-test-170720.zip?download=1’ saved [934724312/934724312]



In [ ]:
!unzip clickbait17-test-170720.zip?download=1

# Setting up model for Webis leaderboard evaluation

In [ ]:
!cat ./clickbait17-test-170720/instances.jsonl

In [ ]:
import pandas as pd
from datasets import load_dataset,concatenate_datasets,Dataset,DatasetDict

df_test = load_dataset('json', data_files='clickbait17-test-170720/instances.jsonl')['train'].to_pandas()
# load_dataset("json", data_files= dataset_base + "clickbait17-train-170331/instances.jsonl")
label_test = load_dataset('json', data_files='clickbait17-test-170720/truth.jsonl')['train'].to_pandas()

# test_dataset = concatenate_datasets([df_test,label_test], axis=1)
df_total_test = df_test.join(label_test.set_index('id'), on='id')

Using custom data configuration default-dd08184332d47b5e
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-dd08184332d47b5e/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

Using custom data configuration default-662db307a1f50fa2
Reusing dataset json (/root/.cache/huggingface/datasets/json/default-662db307a1f50fa2/0.0.0/ac0ca5f5289a6cf108e706efcf040422dbbfa8e658dee6a819f20d76bb84d26b)


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
df_total_test['truthClass'] = pd.factorize(df_total_test['truthClass'])[0]

In [ ]:
import numpy as np
hf_total_test = DatasetDict()
df_total_test['postText'].replace(np.nan,'', inplace=True)
hf_total_test['train'] = Dataset.from_pandas(df_total_test)

In [ ]:
hf_total_test['train']['postText']

[['Johnny Manziel on Browns\' No. 1 pick Myles Garrett: "He\'ll do wonders for the team"\n\n🎥:'],
 ["Fabio: California Is a 'Mess' Because of Liberal Policies"],
 ['"He\'s been huge for us this year, man."'],
 ['New Bears quarterback Mitchell Trubisky was greeted with some boos at his first public appearance.'],
 ["It's not enough to let employees work flexible schedules if they're still afraid to ask"],
 ["Out of 15 lakh registered companies in India, 9 lakh don't file returns!"],
 ['Tense moment plane lands without wheel'],
 ['Stars who got their start on soap operas'],
 ['President Donald Trump: My first 100 days are "just about the most successful" in US history'],
 ['U.S. wants more U.N. sanctions over North Korea’s nuclear arms, warns time is short'],
 ["Don't abandon the snot if you want to be healthy. #EwwButTrue"],
 ['This Goldman trader had one of 2016’s most successful trades. Then things changed'],
 ['DMX reportedly enters rehab'],
 ["Money is pouring into the world's hotte

In [ ]:
from pprint import pprint
def preprocess_function_title_only_classification(examples):
    # pprint(examples['postText'])
    model_inputs = classifier_tokenizer([sample[0] for sample in examples['postText']], padding="max_length", truncation=True, max_length=25)
      
    model_inputs['labels'] = examples['truthClass']

    return model_inputs

hf_total_test = hf_total_test.map(preprocess_function_title_only_classification, batched=True)

  0%|          | 0/19 [00:00<?, ?ba/s]

In [ ]:
from transformers import Trainer
from transformers import TrainingArguments
from tqdm.auto import tqdm
from torch.utils.data import DataLoader
from transformers import DataCollatorWithPadding
from transformers import AdamW,get_scheduler
import torch
from datasets import load_metric

output_path = "drive/MyDrive/nlp_lss_data/"

debug = False

data_collator = DataCollatorWithPadding(classifier_tokenizer)

training_args = TrainingArguments(
    # output_dir=output_path+'clickbait_classification_maxlen25',           # output directory
    output_dir=output_path+'roberta_clickbait_classification_maxlen25',           # output directory
    evaluation_strategy ="epoch",
    save_strategy="epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=64,
    per_device_eval_batch_size=1,
    warmup_steps=100,
    num_train_epochs=6,
    weight_decay=0.01,
    save_total_limit=1,
    load_best_model_at_end=True,
    report_to="none"
)
trainer = Trainer(
    model=classifier_model,
    args=training_args,
    train_dataset=hf_total_test['train'], #webis_total["train"].shuffle(seed=42).select(range(50)) if debug else webis_total["train"].shuffle(seed=42),  
    eval_dataset=hf_total_test['train'], #webis_total["validation"].shuffle(seed=42).select(range(10)) if debug else webis_total["validation"].shuffle(seed=42),
    # compute_metrics=compute_metrics,
    tokenizer=classifier_tokenizer,
    data_collator=data_collator
)

PyTorch: setting up devices


In [ ]:
# ['input_ids', 'labels', 'attention_mask']
remove_columns_list = list(set(hf_total_test['train'].column_names) - set(['input_ids', 'labels', 'attention_mask', 'id']))

In [ ]:
hf_total_test['train']=hf_total_test['train'].remove_columns(remove_columns_list)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight

weight = torch.FloatTensor(compute_class_weight(class_weight="balanced", classes=[0, 1], y=df_total_test["truthClass"])).to(torch.device('cuda'))
WEIGHT = torch.cuda.FloatTensor([weight[1]/weight[0]])

In [ ]:
y_test = trainer.predict(hf_total_test["train"])
y_pred = (y_test.predictions > 0.5).astype(int).ravel()

from sklearn.metrics import classification_report
print(classification_report(y_test.label_ids, y_pred, target_names = ["No-clickbait", "Clickbait"]))




***** Running Prediction *****
  Num examples = 18979
  Batch size = 1
The following columns in the test set don't have a corresponding argument in `BertClickbaitClassifier.forward` and have been ignored: id. If id are not expected by `BertClickbaitClassifier.forward`,  you can safely ignore this message.


              precision    recall  f1-score   support

No-clickbait       0.89      0.92      0.90     14464
   Clickbait       0.71      0.65      0.68      4515

    accuracy                           0.85     18979
   macro avg       0.80      0.78      0.79     18979
weighted avg       0.85      0.85      0.85     18979



In [ ]:
!ls clickbait17-test-170720/truth.jsonl

clickbait17-test-170720/truth.jsonl


In [ ]:
import json
output_format = {}
with open("outputs.jsonl", "w") as pred_file:
  for id, prediction, label in zip(hf_total_test['train']['id'], y_test.predictions,hf_total_test['train']['labels']):
    result = {"id": str(id), "clickbaitScore": float(prediction[0])}

    json.dump(result, pred_file)
    pred_file.write('\n')




In [ ]:
!ls outputs.jsonl

outputs.jsonl


In [ ]:
!cp outputs.jsonl drive/MyDrive/nlp_lss_data/
!cp clickbait17-test-170720/truth.jsonl drive/MyDrive/nlp_lss_data/

## eval.py is file download from Webis website for evaluation

In [ ]:
!cd drive/MyDrive/nlp_lss_data/
!python eval.py drive/MyDrive/nlp_lss_data/truth.jsonl drive/MyDrive/nlp_lss_data/outputs.jsonl drive/MyDrive/nlp_lss_data/output.prototext


Dataset Stats
Size: 18979
#Clickbait: 4515
#No-Clickbait: 14464

Regression scores
Explained variance: -0.1936135432130548
Mean absolute error: 0.2239249845921791
Mean squared error: 0.09321661232078872
Median absolute error: 0.18821836713976203
R2 score: -0.26780100040417554
Normalized mean squared error: 1.2678010004041755

Binary classification scores
Accuracy: 0.8529954159860899
Precision: 0.7077812575283065
Recall: 0.650719822812846
F1 score: 0.6780521578582969

Classification report
              precision    recall  f1-score   support

           0       0.89      0.92      0.90     14464
           1       0.71      0.65      0.68      4515

    accuracy                           0.85     18979
   macro avg       0.80      0.78      0.79     18979
weighted avg       0.85      0.85      0.85     18979



In [ ]:
!ls drive/MyDrive/nlp_lss_data/

 chakraborty
 clickbait17-test-170720
 clickbait17-train-170331
'clickbait17-train-170331.zip?download=1'
 clickbait17-unlabeled-170429
 clickbait17-validation-170630
 clickbait_classification_maxlen25
 clickbait_data
 clickbait_only_summarization
 clickbait_only_summarization_large
 clickbait_only_summarization_steps_large
 eval.py
 fudge
 gpt2_conditional
 grover_model
 logs
 logs_full_text
 logs_full_text_reverse
 mpnet_clickbait_classification_maxlen25
 non_clickbait_data
 outputs.jsonl
 realnews
 results
 results_classifier_reload
 results_full_text
 results_full_text_reverse
 results_taken_classification
 result_summary.csv
 roberta_clickbait_classification_maxlen25
 truth.jsonl
